## Persistence

In [66]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
#db
# from langgraph.checkpoint.sqlite import SqliteSaver

model = ChatOllama(
    model="qwen2.5:14b",
    temperature=0.3, 
    top_p=0.95, 
    num_ctx=2048, 
    repeat_penalty=1.2,
    timeout=120
)
#Define a new graph
workflow = StateGraph(state_schema=MessagesState)
#define a function that calls model
def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages" : response}

#define a node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable" : {"thread_id" : "thread_1"}}


## Here we told him our name

In [67]:
query = "Hi, I am bob"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Bob! It's nice to meet you. How can I assist you today? If you have any questions or need help with anything specific, feel free to let me know.


## He remembers it

In [68]:
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob according to our previous conversation. Is there something specific you'd like to know about names or are you asking a different question?


## Here he should forget our name, since we changed the thread id

In [71]:
config = {"configurable" : {"thread_id" : "thread_2"}}
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but as a language model, I do not have access to个人信息或用户的私人数据。您没有告诉我您的名字，所以我无法知道叫您什么。您可以告诉我是如何称呼您的吗？如果您有其他问题或者需要帮助，请随时告诉我！


### Here, he remembers it again, since we are passing the original thread_1

In [72]:
config = {"configurable" : {"thread_id" : "thread_1"}}
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob according to our conversation. Is there something specific you'd like to ask about names or are you looking for assistance with something else?
